In [ ]:
import os
import cv2
import tensorflow as tf
import numpy as np
from typing import List
from matplotlib import pyplot as plt
import imageio
from typing import Tuple, Union
import math
import mediapipe as mp
from keras import backend as K
import sys
import random
from celluloid import Camera 
import matplotlib.pyplot as plt
from IPython.display import HTML
from moviepy.editor import ImageSequenceClip
from IPython.display import display, Image
from tensorflow.keras.models import Sequential ,model_from_json
from tensorflow.keras.layers import Conv3D,GRU, Dense, Dropout, Bidirectional, MaxPool3D, Activation, TimeDistributed, Flatten, LSTM
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from scipy.ndimage import gaussian_filter
import keras

print(sys.executable)

In [ ]:
# pixels
MARGIN = 0
ROW_SIZE = 0
FONT_SIZE = 1
FONT_THICKNESS = 1

# Input model Image size
IMAGE_SIZE = (100, 200)

batch_size = 1
mp_face_mesh =  mp.solutions.face_mesh.FaceMesh(max_num_faces=1,static_image_mode=True, min_detection_confidence=0.2)
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))


In [ ]:
char_to_num = tf.keras.layers.StringLookup(vocabulary=vocab, oov_token="")
num_to_char = tf.keras.layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), oov_token="", invert=True
)

print(
    f"The vocabulary is: {char_to_num.get_vocabulary()} "
    f"(size ={char_to_num.vocabulary_size()})"
)

In [ ]:
def char_to_arab(alignments):
    sub_arrays = []
    current_sub_array = []
    for element in num_to_char(alignments).numpy():
        if element.decode("utf-8") == "":
            if current_sub_array:  # Add a non-empty sub-array
                sub_arrays.append(current_sub_array)
            current_sub_array = []  # Start a new sub-array
        else:
            current_sub_array.append(element.decode("utf-8"))

    if current_sub_array:  # Add the last sub-array if it's not empty
        sub_arrays.append(current_sub_array)

    for i in sub_arrays:
        print(''.join(i),end=" ")
def normalized_to_pixel_coordinates(normalized_x, normalized_y, image_width, image_height):
    """Converts normalized values to pixel coordinates."""
    x_px = min(max(int(normalized_x * image_width), 0), image_width)
    y_px = min(max(int(normalized_y * image_height), 0), image_height)
    return x_px, y_px

def contrast_based_histogram_equalization_rgb(image):
    # # Convert the image to uint8 if not already
    # if image.dtype != np.uint8:
    #     image = (image * 255).astype(np.uint8)

    # Split channels
    channels = cv2.split(image)

    # Apply histogram equalization to each channel
    equalized_channels = [cv2.equalizeHist(channel) for channel in channels]

    # Merge channels
    equalized_image = cv2.merge(equalized_channels)

    return equalized_image

def clahe_equalization_rgb(image):
    # Convert the image to LAB color space
    lab_image = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)

    # Split LAB image into L, A, B channels
    l_channel, a_channel, b_channel = cv2.split(lab_image)

    # Apply CLAHE to the L channel
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    clahe_l_channel = clahe.apply(l_channel)

    # Merge the enhanced L channel with the original A and B channels
    clahe_lab_image = cv2.merge((clahe_l_channel, a_channel, b_channel))

    # Convert the LAB image back to RGB
    clahe_rgb_image = cv2.cvtColor(clahe_lab_image, cv2.COLOR_LAB2RGB)

    return clahe_rgb_image

def scale_image_0_1(image):

    normalized_image = (image - np.min(image)) / (np.max(image) - np.min(image))

    return normalized_image

def scale_image__1_1(image):

    normalized_image = 2 * (image - np.min(image, axis=(0, 1))) / (np.max(image, axis=(0, 1)) - np.min(image, axis=(0, 1))) - 1
    normalized_image = np.clip(normalized_image, -1, 1)
    
    return normalized_image

def euclidean_distance(point1, point2):
    x1, y1 = point1
    x2, y2 = point2
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def inner_angle(point1, point2, point3):
    a = np.array(point1)
    b = np.array(point2)
    c = np.array(point3)

    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)

    return np.degrees(angle)

def display_gif(video):
    merged_array = np.concatenate((video.numpy(), video.numpy(), video.numpy()), axis=-1)
    print(merged_array.shape)
    fig, ax = plt.subplots() # make it bigger
    camera = Camera(fig)# the camera gets our figure
    for img in os.listdir("NST/epochs"):
        img_obj = plt.imread(os.path.join("NST/epochs"), img) # reading
        ax.imshow(img_obj) # plotting
        camera.snap()
    animation = camera.animate()
    HTML(animation.to_html5_video())
    clip = ImageSequenceClip(list(scale_image_0_1(merged_array)*255), fps=25)
    clip.write_gif('test.gif', fps=25)
    return Image('test.gif')

def use_camera():
    cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

    average_angle = []
    while True:
        ret, frame = cap.read()

        if not ret:
            print("Broken image")
            continue

        # Convert to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Get face mesh landmarks
        results = mp_face_mesh.process(frame_rgb)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                image_rows, image_cols, _ = frame.shape
                normalized_point = normalized_to_pixel_coordinates(1,1, image_cols, image_rows)

                left_lip = face_landmarks.landmark[0]
                right_lip = face_landmarks.landmark[17]
                left = face_landmarks.landmark[192]
                bottom= face_landmarks.landmark[199]
                top = face_landmarks.landmark[2]
                right = face_landmarks.landmark[416]

                bottom_px = normalized_to_pixel_coordinates(bottom.x, bottom.y, image_cols, image_rows)
                left_px = normalized_to_pixel_coordinates(left.x, left.y, image_cols, image_rows)
                top_px = normalized_to_pixel_coordinates(top.x, top.y, image_cols, image_rows)
                right_px = normalized_to_pixel_coordinates(right.x, right.y, image_cols, image_rows)
                left_lip = normalized_to_pixel_coordinates(left_lip.x, left_lip.y, image_cols, image_rows)
                right_lip = normalized_to_pixel_coordinates(right_lip.x, right_lip.y, image_cols, image_rows)


                midpoint = [(left_lip[0]+right_lip[0])//2,(left_lip[1]+right_lip[1])//2]

                # Calculate the differences in x and y coordinates
                dx = right_lip[0] - midpoint[0]
                dy = right_lip[1] - midpoint[1]
                
                # # Calculate the angle using arctan(dy/dx)
                angle_radians = math.atan2(dy, dx)
                
                # # Convert radians to degrees
                angle_degrees = math.degrees(angle_radians) - 90

                if len(average_angle) <6:
                    average_angle.append(angle_degrees)
                else:
                    average_angle.pop(0)
                    average_angle.append(angle_degrees)
                    
                if sum(average_angle) == 0:
                    angle_degrees = 0
                else:
                    angle_degrees = sum(average_angle) / len(average_angle)

                # # Rotate the image
                rotation_matrix = cv2.getRotationMatrix2D(midpoint, angle_degrees, 1.0)
                frame = cv2.warpAffine(frame, rotation_matrix, (normalized_point))

                # Convert rotated points back to list of tuples
                bottom_px , left_px , top_px , right_px = [(int(point[0][0]), int(point[0][1])) for point in cv2.transform(np.array([bottom_px , left_px , top_px , right_px]).reshape(-1, 1, 2).astype(np.float64), rotation_matrix)]

                if not (frame.shape[0] >0 and frame.shape[1] >0 and frame.shape[2] ==3):
                    continue

                frame = frame[
                        top_px[1] : bottom_px[1],
                        left_px[0] : right_px[0],
                        :,
                    ]
                
                if not (frame.shape[0] >0 and frame.shape[1] >0 and frame.shape[2] ==3):
                    continue

                frame = cv2.resize(
                        frame, (IMAGE_SIZE[1],IMAGE_SIZE[0]), interpolation=cv2.INTER_LANCZOS4
                    )
                
                frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)

                # frame = cv2.equalizeHist(frame)
                frame = clahe.apply(frame)
                frame = gaussian_filter(frame, sigma=0.2)

                mean = np.mean(frame, axis=(0, 1))  
                std = np.std(frame, axis=(0, 1))    
                frame = (frame - mean) / std
                cv2.imshow("frame", scale_image_0_1 (frame))
                break

        # Display the frame
        
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()

# use_camera()



In [ ]:
def load_video(path: str) -> List[float]:
    # opens the video's path as a camera object
    cap = cv2.VideoCapture(path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # frames of the video
    frames = np.empty((MAX_FRAMES, IMAGE_SIZE[0], IMAGE_SIZE[1], 1))
    average_angle = []
    idx = 0

    # loop over all frames in the video
    for _ in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
        # if _ == 0:
        #     continue
        try:
            ret, frame = cap.read()
            image_rows, image_cols, _ = frame.shape
            # convert to rgb
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # get the keypoints (x,y)s of each point of a face in the frame
            face_mesh_result = mp_face_mesh.process(frame)

            # loop over each face landmark ~ (nose (x,y) lips (x,y) and so one and a bounding box)
            if face_mesh_result.multi_face_landmarks:

                for face_landmarks in face_mesh_result.multi_face_landmarks:

                    normalized_point = normalized_to_pixel_coordinates(1,1, image_cols, image_rows)

                    left_lip = face_landmarks.landmark[0]
                    right_lip = face_landmarks.landmark[17]
                    left = face_landmarks.landmark[192]
                    bottom= face_landmarks.landmark[199]
                    top = face_landmarks.landmark[2]
                    right = face_landmarks.landmark[416]

                    bottom_px = normalized_to_pixel_coordinates(bottom.x, bottom.y, image_cols, image_rows)
                    left_px = normalized_to_pixel_coordinates(left.x, left.y, image_cols, image_rows)
                    top_px = normalized_to_pixel_coordinates(top.x, top.y, image_cols, image_rows)
                    right_px = normalized_to_pixel_coordinates(right.x, right.y, image_cols, image_rows)
                    left_lip = normalized_to_pixel_coordinates(left_lip.x, left_lip.y, image_cols, image_rows)
                    right_lip = normalized_to_pixel_coordinates(right_lip.x, right_lip.y, image_cols, image_rows)


                    midpoint = [(left_lip[0]+right_lip[0])//2,(left_lip[1]+right_lip[1])//2]

                    # Calculate the differences in x and y coordinates
                    dx = right_lip[0] - midpoint[0]
                    dy = right_lip[1] - midpoint[1]
                    
                    # # Calculate the angle using arctan(dy/dx)
                    angle_radians = math.atan2(dy, dx)
                    
                    # # Convert radians to degrees
                    angle_degrees = math.degrees(angle_radians) - 90

                    if len(average_angle) <6:
                        average_angle.append(angle_degrees)
                    else:
                        average_angle.pop(0)
                        average_angle.append(angle_degrees)
                        
                    if sum(average_angle) == 0:
                        angle_degrees = 0
                    else:
                        angle_degrees = sum(average_angle) / len(average_angle)

                    # Rotate the image
                    rotation_matrix = cv2.getRotationMatrix2D(midpoint, angle_degrees, 1.0)
                    frame = cv2.warpAffine(frame, rotation_matrix, (normalized_point))

                    # Convert rotated points back to list of tuples
                    bottom_px , left_px , top_px , right_px = [(int(point[0][0]), int(point[0][1])) for point in cv2.transform(np.array([bottom_px , left_px , top_px , right_px]).reshape(-1, 1, 2).astype(np.float64), rotation_matrix)]

                    if not (frame.shape[0] >0 and frame.shape[1] >0 and frame.shape[2] ==3):
                        continue

                    frame = frame[
                            top_px[1] : bottom_px[1],
                            left_px[0] : right_px[0],
                            :,
                        ]
                    
                    if not (frame.shape[0] >0 and frame.shape[1] >0 and frame.shape[2] ==3):
                        continue

                    frame = cv2.resize(
                            frame, (IMAGE_SIZE[1],IMAGE_SIZE[0]), interpolation=cv2.INTER_LANCZOS4
                        )

                    # frame = clahe_equalization_rgb(frame)

                    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
                    frame = clahe.apply(frame)
                    frame = gaussian_filter(frame, sigma=0.2)

                        
                    mean = np.mean(frame)  
                    std = np.std(frame)    
                    frame = (frame - mean) / std
                    
                    frames[idx] = frame.reshape((*IMAGE_SIZE,1))

                    idx += 1
                    break

        except Exception as e:
                # Handle the case where the landmarks are not found
            print(e)
            return None

    cap.release()
    cv2.waitKey(0)

    # normalize on the scale of the video
    # mean = tf.math.reduce_mean(frames)
    # std = tf.math.reduce_std(tf.cast(frames, tf.float32))
    # frames = tf.cast((frames - mean), tf.float32) / std
    frames = tf.cast(frames, tf.float32)
    if idx < MAX_FRAMES:
        for i in range(idx, MAX_FRAMES):
            frames = tf.tensor_scatter_nd_update(frames, [[i]], tf.zeros((1, IMAGE_SIZE[0], IMAGE_SIZE[1], 1), dtype=tf.float32))

    # frames = tf.cast(tf.convert_to_tensor(frames),tf.float32)
    
    return frames


In [ ]:
display_gif(load_video(FILTERED_VIDEOS + fr"/{VIDEO_DIR[0]}"))

In [ ]:
char_to_num(["","ا", "ن", "ا", " ", "ا", "ن", "ا", ""])

In [ ]:
char_to_arab([1, 25, 1,0, 1, 25, 1])

In [ ]:
def load_data(path: str): 

    path = bytes.decode(path.numpy())
    
    file_name = path.replace("\\","/").split('/')[-1].split('.')[0]
    video_path = FILTERED_VIDEOS + fr'/{file_name}.mp4'
    alignment_path = FILTERED_ALIGNMENTS + fr'/{file_name}.align'

    frames = load_video(video_path) 
    alignments = load_alignments(alignment_path)
    
    return frames, alignments

In [ ]:
frames, alignments = load_data(tf.convert_to_tensor(test_path))

In [ ]:
display_gif(frames)

In [ ]:
char_to_arab(alignments)

In [16]:
def convert_vtt_to_srt(vtt_file_path, srt_file_path):
    with open(vtt_file_path, 'r', encoding='utf-8') as vtt_file:
        lines = vtt_file.readlines()

    srt_lines = []
    counter = 1

    for line in lines:
        # Skip WebVTT-specific lines
        if line.startswith("WEBVTT") or line.startswith("NOTE"):
            continue
        
        # Replace VTT timestamp comma with SRT timestamp period
        if '-->' in line:
            line = line.replace('.', ',')  # Replace the first period with a comma for SRT format

        # Add counter at the beginning of each subtitle block
        if '-->' in line:
            srt_lines.append(str(counter))
            counter += 1
        
        srt_lines.append(line.strip())

    with open(srt_file_path, 'w', encoding='utf-8') as srt_file:
        srt_file.write('\n'.join(srt_lines))
        srt_file.write('\n')

# Usage
convert_vtt_to_srt(r'D:/Code-Projects/University/Fall2023/Grad/LipNet-Assistive-AI/notebooks/sub.srt.ar.vtt', 'output.srt')


In [14]:
import yt_dlp

url = "https://www.youtube.com/watch?v=bpu2hbpQpCY&pp=ygUM2KfZhNij2LHYr9mG"

ydl_opts = {
  'subtitleslangs': ['ar'],
  'writeautomaticsub': True,
  'subtitlesformat': 'srt',
  'skip_download': True,
  'outtmpl': 'sub.srt' 
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
  ydl.download([url])

[youtube] Extracting URL: https://www.youtube.com/watch?v=bpu2hbpQpCY&pp=ygUM2KfZhNij2LHYr9mG
[youtube] bpu2hbpQpCY: Downloading webpage
[youtube] bpu2hbpQpCY: Downloading ios player API JSON
[youtube] bpu2hbpQpCY: Downloading player 8eff86d5
[youtube] bpu2hbpQpCY: Downloading m3u8 information
[info] bpu2hbpQpCY: Downloading subtitles: ar


[info] bpu2hbpQpCY: Downloading 1 format(s): 303+251
[info] Writing video subtitles to: sub.srt.ar.vtt
[download] Destination: sub.srt.ar.vtt
[download] 100% of   20.20KiB in 00:00:00 at 120.94KiB/s


In [11]:
import yt_dlp

def check_arabic_auto_subtitles(video_url):
    ydl_opts = {
        'listsubtitles': True,
        'skip_download': True,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        result = ydl.extract_info(video_url, download=False)
        subtitles = result.get('subtitles', {})
        auto_subtitles = result.get('automatic_captions', {})
    
        print(auto_subtitles.keys())
# video_url = "https://www.youtube.com/watch?v=bpu2hbpQpCY&pp=ygUM2KfZhNij2LHYr9mG"

video_url = "https://www.youtube.com/watch?v=bpu2hbpQpCY&pp=ygUM2KfZhNij2LHYr9mG"
check_arabic_auto_subtitles(video_url)


[youtube] Extracting URL: https://www.youtube.com/watch?v=bpu2hbpQpCY&pp=ygUM2KfZhNij2LHYr9mG
[youtube] bpu2hbpQpCY: Downloading webpage
[youtube] bpu2hbpQpCY: Downloading ios player API JSON
[youtube] bpu2hbpQpCY: Downloading m3u8 information
[info] Available automatic captions for bpu2hbpQpCY:
Language Name                  Formats
af       Afrikaans             vtt, ttml, srv3, srv2, srv1, json3
ak       Akan                  vtt, ttml, srv3, srv2, srv1, json3
sq       Albanian              vtt, ttml, srv3, srv2, srv1, json3
am       Amharic               vtt, ttml, srv3, srv2, srv1, json3
ar-orig  Arabic (Original)     vtt, ttml, srv3, srv2, srv1, json3
ar       Arabic                vtt, ttml, srv3, srv2, srv1, json3
hy       Armenian              vtt, ttml, srv3, srv2, srv1, json3
as       Assamese              vtt, ttml, srv3, srv2, srv1, json3
ay       Aymara                vtt, ttml, srv3, srv2, srv1, json3
az       Azerbaijani           vtt, ttml, srv3, srv2, srv1, json3
bn  

In [1]:
from pytube import YouTube

In [2]:
yt = YouTube(r"https://www.youtube.com/watch?v=bpu2hbpQpCY&pp=ygUM2KfZhNij2LHYr9mG")

In [3]:
yt.captions.keys()

KeysView({})

In [8]:
yt = YouTube(r"https://www.youtube.com/watch?v=lPrjP4A_X4s")
# stream = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()

"a.ar" in yt.captions.keys()

False